# TWS API DEMO

+ ensure that you have TWS and running, API enabled etc... (see README.md)
+ market data subscriptions are not needed as live prices are extracted from yahoo finance
+ this example is *self-contained* meaning that it doesn't use any other .py files or anything
+ you can run it if you have installed dependencies and enabled api (as specified in README.md)

In [1]:
import yfinance as yf
import math

In [2]:
from ib_insync import *
util.startLoop()

ib = IB()

In [3]:
# ensure that we connect just once
if not ib.isConnected():
    print('connecting')
    ib.connect('127.0.0.1', 7497, clientId=0, readonly=False)
    if ib.isConnected():
        ib.reqMarketDataType(1)
        print('connected')
    else:
        print('Error, try again or smthing')
else:
    print('already connected')


connecting
connected


In [4]:
# todo - move to utils, add currency and autoconversion
# port over the ib price getting
def get_current_price(ticker, yahoo=True):
    if yahoo:
        # todo handle none, wrong ticker etc
        t = yf.Ticker(ticker)
        return t.info["regularMarketPrice"]
    else:
        raise NotImplemented("This feature is not implemented yet")

In [5]:
pos = ib.positions()

# it is assumed that available funds are in eur
cash_to_burn = [v for v in ib.accountValues() if v.tag == 'AvailableFunds' and v.currency == 'EUR'][0].value
cash_to_burn = float(cash_to_burn) - 50 # we want to have some money left


In [6]:
def round_down(x, a):
    return math.floor(x / a) * a

def round_nearest(x, a):
    return round(x / a) * a

In [7]:
vwce_price = get_current_price('VWCE.DE')
vwce_to_buy = math.floor(cash_to_burn / vwce_price)
print(f'Going to buy {vwce_to_buy} VWCE at {vwce_price}')

Going to buy 8483 VWCE at 104.48


In [8]:
ib_contract = Stock("VWCE", "SMART", "EUR")
ib.qualifyContracts(ib_contract)

# round price to nearest multiple of 0.05, bc exchange requires it and TWS ain't gonna do it for ourselves
# it might vary between securities
if vwce_to_buy < 1:
    print("You don't have enough to buy a share of VWCE. What a looser!")
order = LimitOrder('BUY', vwce_to_buy, round_nearest(vwce_price * 1.03, 0.05))
trade = ib.placeOrder(ib_contract, order)

ib.sleep(5)
trade.log

[TradeLogEntry(time=datetime.datetime(2021, 11, 21, 10, 28, 23, 658859, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2021, 11, 21, 10, 28, 23, 845653, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2021, 11, 21, 10, 28, 23, 913200, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0)]

In [9]:
ib.disconnect()
